# Deploying AI
## Assignment 1: Evaluating Summaries

A key application of LLMs is to summarize documents. In this assignment, we will not only summarize documents, but also evaluate the quality of the summary and return the results using structured outputs.

## Select a Document

Please select one out of the following articles:

+ [Managing Oneself, by Peter Druker](https://www.thecompleteleader.org/sites/default/files/imce/Managing%20Oneself_Drucker_HBR.pdf)  (PDF)
+ [The GenAI Divide: State of AI in Business 2025](https://www.artificialintelligence-news.com/wp-content/uploads/2025/08/ai_report_2025.pdf) (PDF)
+ [What is Noise?, by Alex Ross](https://www.newyorker.com/magazine/2024/04/22/what-is-noise) (Web)

**Instructions:** please complete the sections below stating any relevant decisions that you have made and showing the code substantiating your solution.

# Load Secrets

In [1]:
%load_ext dotenv
%dotenv ../05_src/.secrets

## Load Document

Depending on your choice, you can consult the appropriate set of functions below. Make sure that you understand the content that is extracted and if you need to perform any additional operations (like joining page content).

### PDF

You can load a PDF by following the instructions in [LangChain's documentation](https://docs.langchain.com/oss/python/langchain/knowledge-base#loading-documents). Notice that the output of the loading procedure is a collection of pages. You can join the pages by using the code below.

```python
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"
```

### Web

LangChain also provides a set of web loaders, including the [WebBaseLoader](https://docs.langchain.com/oss/python/integrations/document_loaders/web_base). You can use this function to load web pages.

In [2]:
# Import PyPDFLoader from langchain community document loaders
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
# Load PDF from url
# pickup "The GenAI Divide: State of AI in Business 2025"
book_url = 'https://www.artificialintelligence-news.com/wp-content/uploads/2025/08/ai_report_2025.pdf'

pdf_loader = PyPDFLoader(book_url)
docs = pdf_loader.load()

In [4]:
# Verify load docs result
print("Type:", type(docs))
print(len(docs))
print(docs[0].page_content[:100])

Type: <class 'list'>
26
pg. 1 
 
 
The GenAI Divide  
STATE OF AI IN 
BUSINESS 2025 
 
 
 
 
 
 
MIT NANDA 
Aditya Challapal


In [5]:
# Join all pages into one string
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"

In [6]:
# Verify document_text
print("Type:", type(document_text))
print("Total characters:", len(document_text))

Type: <class 'str'>
Total characters: 53851


## Generation Task

Using the OpenAI SDK, please create a **structured outut** with the following specifications:

+ Use a model that is NOT in the GPT-5 family.
+ Output should be a Pydantic BaseModel object. The fields of the object should be:

    - Author
    - Title
    - Relevance: a statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development.
    - Summary: a concise and succinct summary no longer than 1000 tokens.
    - Tone: the tone used to produce the summary (see below).
    - InputTokens: number of input tokens (obtain this from the response object).
    - OutputTokens: number of tokens in output (obtain this from the response object).
       
+ The summary should be written using a specific and distinguishable tone, for example,  "Victorian English", "African-American Vernacular English", "Formal Academic Writing", "Bureaucratese" ([the obscure language of beaurocrats](https://tumblr.austinkleon.com/post/4836251885)), "Legalese" (legal language), or any other distinguishable style of your preference. Make sure that the style is something you can identify. 
+ In your implementation please make sure to use the following:

    - Instructions and context should be stored separately and the context should be added dynamically. Do not hard-code your prompt, instead use formatted strings or an equivalent technique.
    - Use the developer (instructions) prompt and the user prompt.


In [7]:
# Step 1: setup OpenAI client
import os
from openai import OpenAI

client = OpenAI(base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1', 
                api_key='any value',
                default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')})

response = client.responses.create(
    model = 'gpt-4o-mini',
    input = 'Hello world!'
    
)

print(response.output_text)

Hello! How can I assist you today?


In [8]:
# Step 2: Define Pydantic Output class
from pydantic import BaseModel, Field

class ArticleStructuredOutput(BaseModel):
    Author: str
    Title: str
    Relevance: str = Field(description="A statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development.")
    Summary: str = Field(description="A concise and succinct summary no longer than 1000 tokens.")
    Tone: str = Field(description="The tone used to produce the summary (see below).")
    InputTokens: int | None = Field(None, description="Number of input tokens (obtain this from the response object).")
    OutputTokens: int | None = Field(None, description="Number of tokens in output (obtain this from the response object).")


In [9]:
# Step 3: Set Tone, developer_instructions, and user_prompt
# Make the Tone is more IT-industry friendly, practical, and easy to understand.

TONE = "Practical IT Industry Explainer"

developer_instructions = f"""
You are an expert technical editor producing structured outputs.

Return output strictly matching the provided schema.

Rules:
- Relevance must be ONE paragraph (no bullet points or lists).
- Summary must be concise, clear, and easy for IT professionals to understand.
- Write the summary using the tone: {TONE}.
- Tone field must EXACTLY equal: {TONE}.
- If author is not explicitly stated, use the publishing organization as Author.
"""

user_prompt_template = """
Context (full report text):
{context}

Task:
Extract the report Title and Author, then write:
1) Relevance → one paragraph explaining why this report matters for AI professionals' career development.
2) Summary → clear, concise summary (<=1000 tokens) in the specified tone.
"""

# Limit context if needed (helps avoid token overflow)
MAX_CHARS = 600_000
context = document_text[:MAX_CHARS]

user_prompt = user_prompt_template.format(context=context)


In [ ]:
# Step 4: Call OpenAI Responses API with Structured Outputs


# Evaluate the Summary

Use the DeepEval library to evaluate the **summary** as follows:

+ Summarization Metric:

    - Use the [Summarization metric](https://deepeval.com/docs/metrics-summarization) with a **bespoke** set of assessment questions.
    - Please use, at least, five assessment questions.

+ G-Eval metrics:

    - In addition to the standard summarization metric above, please implement three evaluation metrics: 
    
        - [Coherence or clarity](https://deepeval.com/docs/metrics-llm-evals#coherence)
        - [Tonality](https://deepeval.com/docs/metrics-llm-evals#tonality)
        - [Safety](https://deepeval.com/docs/metrics-llm-evals#safety)

    - For each one of the metrics above, implement five assessment questions.

+ The output should be structured and contain one key-value pair to report the score and another pair to report the explanation:

    - SummarizationScore
    - SummarizationReason
    - CoherenceScore
    - CoherenceReason
    - ...

# Enhancement

Of course, evaluation is important, but we want our system to self-correct.  

+ Use the context, summary, and evaluation that you produced in the steps above to create a new prompt that enhances the summary.
+ Evaluate the new summary using the same function.
+ Report your results. Did you get a better output? Why? Do you think these controls are enough?

Please, do not forget to add your comments.


# Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

## Submission Parameters

- The Submission Due Date is indicated in the [readme](../README.md#schedule) file.
- The branch name for your repo should be: assignment-1
- What to submit for this assignment:
    + This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
- What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    + Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

## Checklist

+ Created a branch with the correct naming convention.
+ Ensured that the repository is public.
+ Reviewed the PR description guidelines and adhered to them.
+ Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
